# 3.1 Churn Prediction Project

เราจะสร้าง ML model สำหรับการทำนายว่าลูกค้าของบริษัท telecom จะมีโอกาสเปลี่ยนไปใช้บริการของคู่แข่งหรือไม่ (Churn) โดยใช้ข้อมูลของลูกค้า เช่น อัตราค่าบริการที่จ้ายอยู่

Source: ข้อมูลจาก [Kaggle](https://www.kaggle.com/blastchar/telco-customer-churn)

หากเราสามารถระบุได้ว่าลูกค้าคนไหนมีแนวโน้มที่จะย้ายค่าย ฝ่ายขายสามารถติดต่อลูกค้าเพื่อเสนอส่วนลดได้ แต่บริษัทก็ไม่อยากที่จะเสนอส่วนลดกับลูกค้าที่ยังไงก็ไม่เปลี่ยนใจ

ตัวแปร target คือ binary คือ ลูกค้าตัดสินใจเปลี่ยนผู้ให้บริการหรือไม่ {0 คือ เปลี่ยน 1 คือ ไม่เปลี่ยน}

Outline:
- Data preparation
- Exploratory data analysis
- Linear regression
- Evaluting the model with RMSE
- Feature engineering
- Regularization

โหลด packages

In [60]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
# train test split
from sklearn.model_selection import train_test_split
# mutal information
from sklearn.metrics import mutual_info_score
# one-hot coding
from sklearn.feature_extraction import DictVectorizer
# train logistic regression
from sklearn.linear_model import LogisticRegression

%matplotlib inline

# 3.2 Data Preparation

- load data
- clean data
- split data

Download ข้อมูล สำหรับคนที่ยังไม่มีข้อมูล

In [ ]:
# URL = "https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv"
# df = pd.read_csv(URL)
# df.to_csv("data/car_price.csv", index=False)

In [3]:
df = pd.read_csv("data/telco_customer_churn.csv")
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [6]:
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [7]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [8]:
# totalcharge should be numeric, we convert with force and fill na with 0
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)

In [9]:
# transform column names to snake case
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [10]:
# transform string columns to snake case
string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [15]:
# convert churn
df.churn = (df.churn == 'yes').astype(int)

In [16]:
df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


# 3.3 Setting Validation Framework

ขั้นตอนต่อมาเป็นการแยกข้อมูลเป็น train validation test set

- split full dataset into full_train and test set
- split full_train set into train and validation set
- sperate X and y

ใช้ function `train_test_split` จาก scikit-learn โดยมี argument หลัก

- test_size = สัดส่วนที่ต้องการแยกไว้ทดสอบ
- random_state = random seed เพื่อให้การสุ่มได้ผลเหมือนเดิม

In [22]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [23]:
df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=11)

In [24]:
# sperate X and y
y_train = df_train.churn.values
y_val = df_val.churn.values

del df_train['churn']
del df_val['churn']

# 3.4 EDA

- missing values
- proportion of target variable

In [27]:
df_train_full.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [28]:
df_train_full.churn.value_counts()

0    4113
1    1521
Name: churn, dtype: int64

In [29]:
global_mean = df_train_full.churn.mean()
round(global_mean, 3)

0.27

In [30]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']

numerical = ['tenure', 'monthlycharges', 'totalcharges']

# 3.5 Feature Importance: Churn Rate and Risk Ratio

ตรวจสอบเบื้องต้นว่าตัวแปร categorial ส่งผลให้เกิดความแตกต่างของ target หรือไม่ สามารถวัดได้จากผลต่าง (diff) ของ churn rate กับค่าเฉลี่ย หรือจะเทียบ สัดส่วนกับค่าเฉลี่ย (risk ratio) เช่น churn rate ของเพศชายและเพศหญิง แตกต่างกันมากหรือไม่

- ใช้ method `groupby` ในการทำ summarize จำแนกตามข้อมูลตาม column ที่กำหนด
- ใช้ method `agg` ในการกำหนด function ในการ summarize เช่น mean

In [31]:
# gender ผลไม่แตกต่างกันมา
female_mean = df_train_full[df_train_full.gender == 'female'].churn.mean()
print('gender == female:', round(female_mean, 3))

male_mean = df_train_full[df_train_full.gender == 'male'].churn.mean()
print('gender == male:  ', round(male_mean, 3))

gender == female: 0.277
gender == male:   0.263


In [36]:
# diff
print('diff')
print('gender == female:', round(female_mean - global_mean, 5))
print('gender == male:', round(male_mean - global_mean, 5))

diff
gender == female: 0.00686
gender == male: -0.00675


In [35]:
# risk ratio
print('risk ratio')
print('gender == female:', round(female_mean / global_mean, 5))
print('gender == male:', round(male_mean / global_mean, 5))

risk ratio
gender == female: 1.0254
gender == male: 0.97498


In [38]:
# combine into dataframe
df_group = df_train_full.groupby(by='gender').churn.agg(['mean'])
df_group['diff'] = df_group['mean'] - global_mean
df_group['risk'] = df_group['mean'] / global_mean
df_group

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


In [37]:
# loop for all categorical variables
for col in categorical:
    df_group = df_train_full.groupby(by=col).churn.agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['risk'] = df_group['mean'] / global_mean
    display(df_group)

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


,mean,diff,risk
seniorcitizen,,,
0,0.242270,-0.027698,0.897403
1,0.413377,0.143409,1.531208


,mean,diff,risk
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


,mean,diff,risk
dependents,,,
no,0.313760,0.043792,1.162212
yes,0.165666,-0.104302,0.613651


,mean,diff,risk
phoneservice,,,
no,0.241316,-0.028652,0.893870
yes,0.273049,0.003081,1.011412


...

,mean,diff,risk
streamingtv,,,
no,0.342832,0.072864,1.269897
no_internet_service,0.077805,-0.192163,0.288201
yes,0.302723,0.032755,1.121328


,mean,diff,risk
streamingmovies,,,
no,0.338906,0.068938,1.255358
no_internet_service,0.077805,-0.192163,0.288201
yes,0.307273,0.037305,1.138182


,mean,diff,risk
contract,,,
month-to-month,0.431701,0.161733,1.599082
one_year,0.120573,-0.149395,0.446621
two_year,0.028274,-0.241694,0.104730


,mean,diff,risk
paperlessbilling,,,
no,0.172071,-0.097897,0.637375
yes,0.338151,0.068183,1.252560


,mean,diff,risk
paymentmethod,,,
bank_transfer_(automatic),0.168171,-0.101797,0.622928
credit_card_(automatic),0.164339,-0.105630,0.608733
electronic_check,0.455890,0.185922,1.688682
mailed_check,0.193870,-0.076098,0.718121


# 3.6 Feature importance: mutal information

- ใช้ function `mutal_info_score` จาก scikit-learn
- ใช้ method `apply` ในการใช้ function `mutal_info_score` ในทุก column ที่เป็น categorical

In [42]:
def calculate_mi(series):
    return mutual_info_score(series, df_train_full.churn)

df_mi = df_train_full[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')

display(df_mi.head())
display(df_mi.tail())

,MI
contract,0.098320
onlinesecurity,0.063085
techsupport,0.061032
internetservice,0.055868
onlinebackup,0.046923


,MI
partner,0.009968
seniorcitizen,0.009410
multiplelines,0.000857
phoneservice,0.000229
gender,0.000117


# 3.7 Feature importance: correlation

เป็นการดูทิศทางของตัวแปร numerical กับ target โดยใช้
- ค่า correlation มีค่าระหว่าง -1 ถึง 1
    - ค่าลบ หมายถึง ความสัมพันธ์ในทิศทางตรงข้าม
    - ค่าบวก หมายถึง ความสัมพันธ์ในทิศทางเดียวกัน
    - ค่าใกล้ 1 หรือ -1 มาก หมายความว่า ระดับความสัมพันธ์มาก
    - ทั้งนี้ correlation ไม่ได้เป็นการบอกความสัมพันธ์เชิงเหตุผล
- ค่าเฉลี่ยของตัวแปร numerical groupby target

In [43]:
df_train_full[numerical].corrwith(df_train_full.churn).to_frame('correlation')

,correlation
tenure,-0.351885
monthlycharges,0.196805
totalcharges,-0.196353


In [44]:
df_train_full.groupby(by='churn')[numerical].mean()

,tenure,monthlycharges,totalcharges
churn,,,
0,37.531972,61.176477,2548.021627
1,18.070348,74.521203,1545.689415


# 3.8 One-Hot Coding

เป็นการแปลงตัวแปร categorical ให้อยู่ในรูปแบบที่แบบจำลอง ML สามารถนำไปวิเคราะห์ โดยการแยก ค่าที่เป็นไปได้ของ categorical variable เป็นแต่ละ column โดยข้อมูลเป็น category ไหน ให้ใส่ค่าเป็น 1 column ที่เหลือ จะเป็น 0 ยกตัวอย่างเช่น column gender จะถูกแยกออกเป็น female และ male โดยที่ ข้อมูลลูกค้าที่เป็นผู้ชาย จะลง code ว่า female = 0, male = 1 เป็นต้น

ในบทก่อน จะเขียน code เอง แต่ในบทนี้จะใช้ function `DictVectorizer` จาก scikit-learn ในการทำงาน

In [46]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')

In [51]:
train_dict[0]

{'gender': 'male',
 'seniorcitizen': 0,
 'partner': 'yes',
 'dependents': 'no',
 'phoneservice': 'yes',
 'multiplelines': 'no',
 'internetservice': 'dsl',
 'onlinesecurity': 'yes',
 'onlinebackup': 'yes',
 'deviceprotection': 'yes',
 'techsupport': 'yes',
 'streamingtv': 'yes',
 'streamingmovies': 'yes',
 'contract': 'two_year',
 'paperlessbilling': 'yes',
 'paymentmethod': 'bank_transfer_(automatic)',
 'tenure': 71,
 'monthlycharges': 86.1,
 'totalcharges': 6045.9}

In [52]:
# สร้าง object dv สำหรับการ transform กับ validation/test set
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

DictVectorizer(sparse=False)

In [54]:
X_train = dv.transform(train_dict)
X_train.shape

(3774, 45)

In [56]:
X_train

array([[0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 1.00000e+00,
        7.10000e+01, 6.04590e+03],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        6.00000e+01, 6.02900e+03],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        4.60000e+01, 2.06515e+03],
       ...,
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        2.00000e+00, 2.83000e+01],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        2.30000e+01, 4.70600e+02],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        6.40000e+01, 5.32725e+03]])

In [59]:
list(dv.get_feature_names_out())

['contract=month-to-month',
 'contract=one_year',
 'contract=two_year',
 'dependents=no',
 'dependents=yes',
 'deviceprotection=no',
 'deviceprotection=no_internet_service',
 'deviceprotection=yes',
 'gender=female',
 'gender=male',
 'internetservice=dsl',
 'internetservice=fiber_optic',
 'internetservice=no',
 'monthlycharges',
 'multiplelines=no',
 'multiplelines=no_phone_service',
 'multiplelines=yes',
 'onlinebackup=no',
 'onlinebackup=no_internet_service',
 'onlinebackup=yes',
 'onlinesecurity=no',
 'onlinesecurity=no_internet_service',
 'onlinesecurity=yes',
 'paperlessbilling=no',
 'paperlessbilling=yes',
 'partner=no',
 'partner=yes',
 'paymentmethod=bank_transfer_(automatic)',
 'paymentmethod=credit_card_(automatic)',
 'paymentmethod=electronic_check',
 'paymentmethod=mailed_check',
 'phoneservice=no',
 'phoneservice=yes',
 'seniorcitizen',
 'streamingmovies=no',
 'streamingmovies=no_internet_service',
 'streamingmovies=yes',
 'streamingtv=no',
 'streamingtv=no_internet_servic

# 3.9 Logistic Regression

# 3.10 Training Logistic Regression with scikit-learn

ใช้ scikit-learn ในการ fit model โดยมีหลักการดังนี้

- สร้าง object
    - เลือก function ของ ml algorithm ในที่นี้คือ `LogisticRegression`
    - กำหนดรูปแบบของแบบจำลอง เช่น `solver`, `random_state` หรือ `C`
    - สร้าง object แล้ว save เป็นตัวแปร เช่น model
- fit model
    - โดยกำหนดข้อมูล X และ y สำหรับ train




In [61]:
# fit model กับ train set
model = LogisticRegression(solver='liblinear', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1, solver='liblinear')

In [62]:
# predict กับ
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [63]:
model.predict_proba(X_val)

array([[0.76508893, 0.23491107],
       [0.7311339 , 0.2688661 ],
       [0.6805482 , 0.3194518 ],
       ...,
       [0.94274725, 0.05725275],
       [0.38476961, 0.61523039],
       [0.93872737, 0.06127263]])

In [65]:
y_pred = model.predict_proba(X_val)[:, 1]
y_pred

array([0.23491107, 0.2688661 , 0.3194518 , ..., 0.05725275, 0.61523039,
       0.06127263])

In [66]:
churn = y_pred > 0.5
(y_val == churn).mean()

0.8016129032258065

# 3.11 Model Interpretation

# 3.12 Using Model

# 3.13 Summary